# 第三节 实践Chain的应用

### 课程笔记大纲

1. LLM链
2. 顺序链
3. 路由器链
4. 组合应用

### 环境安装配置

In [ ]:
%pip install --upgrade --quiet openai langchain-openai langchain

In [ ]:
import os
API_SECRET_KEY = "填写【个人资料】中获取的最新token"
BASE_URL = "https://www.ai360labs.com/openai/v1/"

os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_BASE_URL"] = BASE_URL

In [ ]:

import openai
from langchain_openai import ChatOpenAI,OpenAI

LangChain最重要的关键构建块，即链（chain）。链通常结合了一个大型语言模型（LLM）和一个提示（prompt），使用这个构建块，你可以将多个这样的构建块组合在一起，对你的文本或其他数据进行一系列操作。

## LLM链介绍 
LLM链是一种基于大型语言模型（LLM）的链式结构，用于对文本或其他数据进行一系列操作。它的主要作用是通过结合LLM和提示，生成对输入数据进行处理的输出结果。

这是一个简单但非常强大的链，它是我们未来将要讨论的许多链的基础。因此，我们将导入三样不同的东西。我们将导入OpenAI模型，也就是LLM。我们将导入聊天提示模板。这是提示。然后我们将导入LLM链。

In [ ]:
from langchain.prompts import ChatPromptTemplate # prompt
from langchain.chains import LLMChain # chain

首先，我们将要做的是初始化我们想要使用的语言模型。所以我们将初始化chat OpenAI，设置一个高温度，这样我们就能得到一些有趣的描述。

In [ ]:
llm = ChatOpenAI(temperature=0.9)

使用提示词模板类构建提问的提示词模板。{product}为占位符，接受外部输入，组合为完整的提示词。

现在我们将初始化一个提示。这个提示将接受一个名为“product”的变量，它将要求LLM生成描述制造该产品的公司最fashion的名称。

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "能否给公司的 {product} 产品取一个fashion的名字?"
)

实例化一个LLMChain，设置LLM和提示词模板。

我们将这两样东西组合成一个链。


LLM链，它非常简单，只是LLM和提示的结合。

这个链将让我们以序列的方式通过提示，进入LLM。

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

运行LLMChain，提问LLM,"能否给公司的人工智能穿戴产品取一个fashion的名字？"

In [ ]:
product = "人工智能穿戴"
chain.run(product)

你可以输入任何你想要的产品描述，并且你可以看到链作为结果输出了什么。

In [ ]:
product = "一款帅气的帽子"
chain.run(product)

## 顺序链

顺序链会一个接一个地运行一系列的链。

顺序链是一种链式结构，用于按照顺序执行一系列的操作或处理步骤。它的主要作用是将多个处理步骤有序地连接起来，以完成复杂的文本或数据处理任务。顺序链通常用于需要按照特定顺序执行多个操作的场景。

环境安装配置后，导入 SimpleSequentialChain 类。

In [ ]:
from langchain.chains import SimpleSequentialChain

需要初始化一个适用的语言模型，通常是一个大型的预训练语言模型，如OpenAI的模型。


In [ ]:
llm = ChatOpenAI(temperature=0.9)

### 简单顺序链（SimpleSequentialChain）

简单顺序链是一种处理单一输入和单一输出的链式结构。它按照固定的顺序执行一系列处理步骤，并将单一输入依次传递给每个步骤，最终生成单一输出结果。

现在假设老板给你一个任务，给公司产品取名字，然后为你们公司写一个简介。任务分为两个步骤，先要取名，后写描述。我们将这种任务顺序通过顺序链的方式表达清楚。

顺序链更像是一种语言表达，先做什么后做什么。

首先定义第一个任务的提示词模板，这些提示显示第一个任务是为公司的产品取名字。

In [ ]:


# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

将语言模型和提示结合形成链，实例化第一个LLMChain，设置LLM和提示词模板。

这个提示将接受产品，并返回描述该公司的最佳名称。这将是第一个链。

In [ ]:
# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

创建第二个链。在这第二个链中，我们将接受公司名称，然后输出该公司的20字描述。

In [ ]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

运行顺序链，将前面2个LLMChain以数组的方式，设置为顺序链的chains属性。顺序链会按照固定的顺序执行一系列处理步骤，并将单一输入依次传递给每个步骤，最终生成单一输出结果。

想象这些链可能希望一个接一个地运行，其中第一个链的输出，公司名称，然后被传递到第二个链中。

我们可以通过创建一个简单的顺序链来轻松做到这一点，其中描述了这两个链。

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

现在，你可以做的是，用任何产品描述运行这个链。

In [ ]:
product = "一款帅气的帽子"

设置产品的名称传递给顺序链，run 一下看看结果是不是符合任务的要求。

In [ ]:
overall_simple_chain.run(product)

chain运行后，先输出了一个公司名称，后又输出了公司的简介。

## 普通顺序链

普通顺序链是一种处理多个输入和多个输出的链式结构。与简单顺序链不同，普通顺序链可以处理多个输入，并生成多个输出结果。这种类型的顺序链通常用于更复杂的数据处理任务，其中输入和输出可能具有多个维度或特征。


先导入普通顺序链，实例化一个ChatOpenAI。

In [ ]:
from langchain.chains import SequentialChain
llm = ChatOpenAI(temperature=0.9)

接下来，以一个实际案例，展现普通顺序链的工作流程。

创建一系列我们将要一个接一个使用的链。我们将使用上面的数据，它有一个评论。

设置第一个提示词模板和第一个LLMChain。第一个链，我们将接受评论{Review}并将其翻译成英语。

第一次接收外部的参数 `Review `,结果导出后保存为 `English_Review `。

In [ ]:


# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

设置第二个提示词模板和第二个LLMChain。第二个链利用第一个链的结果，我们将创建该评论的一句话摘要。

接收外部的参数 `English_Review `,结果导出后保存为 `summary `。

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

设置第三个提示词模板和第三个LLMChain。第三个链将检测评论最初是用哪种语言写的。

接收外部的参数 `Review `,结果导出后保存为 `language`。

In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

第四个链将接受多个输入。

设置第四个提示词模板和第四个LLMChain。

因此，这将接受我们在第二个链中计算出的summary变量，以及我们在第三个链中计算出的language变量。

它将要求用指定的语言对摘要做出回应。

接收外部的参数 `summary ` 和  `language `,结果导出后保存为 `followup_message`。

In [ ]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

所有这些子链需要非常精确输入键和输出键。

使用 SequentialChain 整合前面四个LLMChain。

链中的任何一步都可以接受多个输入变量。当你有更复杂的下游链需要多个先前链的组合时，这是非常有用的。现在我们有了所有这些链，我们可以轻松地将它们组合在顺序链中。

四个链传递到chains变量中。我们将创建一个输入变量，这是一个人类输入，即评论。还需要返回所有中间输出。

In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

我们给一个外部输入 `review`后，运行该顺序链。

In [ ]:
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
overall_chain(review)

LLM输出{'Review': "省略",
 'English_Review': "省略",
 'summary': '省略',
 'followup_message': "省略"}

可以在这里看到，原始评论看起来是用法语写的。我们可以看到英语评论作为翻译。我们可以看到该评论的摘要，然后我们可以看到用原始语言法语写的后续消息。

## 路由器链介绍 (RouterChain)

路由器链是一种用于根据输入的类型将输入路由到不同的子链的链式结构。它的主要作用是根据输入的内容或特征将输入数据定向到不同的处理子链，从而实现针对不同类型的输入进行个性化处理的目的。

### 路由器链案例

假设学校开设四门课，物理、数学、计算机科学、历史，分别是四个教授担任讲师。当我们提问时，由导学台（LLM）判断问题的类型，分配不同的老师，由专业的老师（这里指的是LLM)回答问题。 

接下来我们看如何运用路由链实现这个需求。


  1. 定义四个不同类型的提示模板：
  2. 导入多提示链，用于在多个不同的提示模板之间进行路由。导入LLM路由器链，利用语言模型本身在不同的子链之间进行路由。导入路由器输出解析器，解析LLM输出为字典，以便确定使用哪个链及该链的输入。
  3. 导入并定义将要使用的语言模型。
  4. 创建目的地链，用于路由器链调用，每个目的地链本身都是一个语言模型链。
  5. 定义默认链，当路由器无法决定使用哪个子链时调用，通常用于处理与给定提示模板无关的输入问题。
  6. 定义LLM模板，用于在不同链之间进行路由，包括任务指令和输出格式。
  7. 构建路由器链，将各部分整合在一起。

第一个提示适合回答物理问题。第二个提示适合回答数学问题，第三个用于历史，然后第四个用于计算机科学。

让我们定义所有这些提示模板。在我们提供了这些提示模板之后，我们可以提供更多关于它们的信息。

我们可以给每一个都起一个名字，然后描述一下。比如这个物理的描述是适合回答关于物理的问题。

1. 定义不同类型的提示模板：

首先新建候选课程的提示模板。他们接收学生的问题，回答该专业下的知识。

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

组合为一个大的数组，方便遍历管理。

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

  2. 导入多提示链，用于在多个不同的提示模板之间进行路由。导入LLM路由器链，利用语言模型本身在不同的子链之间进行路由。导入路由器输出解析器，解析LLM输出为字典，以便确定使用哪个链及该链的输入。

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

3. 定义将要使用的语言模型，“给对话机器人注入说话的能力。”

In [ ]:
llm = ChatOpenAI(temperature=0)

4. 创建目的地链，用于路由器链调用，每个目的地链本身都是一个语言模型链。

这四个学科的LLMChain，将所有的学科的提示词名称和描述以json字符串的形式拼接为 `destinations_str`变量。

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

5. 定义默认链，当路由器无法决定使用哪个子链时调用，通常用于处理与给定提示模板无关的输入问题。

在上面的例子中，当输入问题与物理、数学、历史或计算机科学无关时，这可能会被调用。

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

为LLMRouterChain，创建一个提示词模板。该模板是理解路由链的关键，该链利用语言模型本身在不同的子链之间进行路由。

它描述了一个多路由模板，目的是根据原始文本输入选择最适合的模型提示。任务包括以下几个要点：

1. 给定原始文本输入到语言模型中，选择最适合输入的模型提示。
2. 提供可用提示的名称和提示适用情况的描述。
3. 如果认为修改原始输入可以导致更好的响应，则可以对原始输入进行修改。
4. 返回一个Markdown代码片段，其中包含格式化为JSON对象的内容。

下面是任务模板中的详细说明：

- 返回的JSON对象应包含两个字段：
  - "destination"：指示要使用的模型提示的名称，或如果输入不适合任何候选提示，则为"DEFAULT"。
  - "next_inputs"：可能是原始输入的修改版本，如果不认为需要任何修改，则可以是原始输入。

任务模板还包括了候选提示的名称和输入/输出的占位符。

6. 定义LLM模板，用于在不同链之间进行路由，包括任务指令和输出格式。

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to wrap the output with ```json (output)```)>>"""

将所有的候选学科模板名称和描述注入到导学台的提示模板占位符中，形成由导学台（LLM）选择走哪个候选学科模板的完整提示词。

实例化 LLMRouterChain，并且配置提示词模板的输入和输出配置，提示词模板使用导学台的模板。

注意：这里我们有路由器输出解析器。这很重要，因为它将帮助这个链决定在哪些子链之间进行路由。

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

7. 最后，把所有东西放在一起，我们可以创建整体链。这有一个路由器链，在这里定义。它有目的地链，我们在这里传入。然后我们还传入默认链。

实例化 MultiPromptChain，配置`router_chain` (导学台）、`destination_chains` （4个候选学科）以及 `default_chain`(学生提问的输入） 。将学生输入问题，导学台分配老师的业务流程串联一体。

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

我们现在可以使用这个链。

我们以不同的学科问题，测试该路由链的完成情况。针对不同类型的学科问题输入，路由链是否定位了正确的候选老师进行个性化处理，使得学生的问题能够得到更加完美的回答。

如果我们问一个关于物理的问题，我们应该希望看到它被路由到物理链，输入是什么是黑体辐射？

In [ ]:
chain.run("What is black body radiation?")

LLM输出：physics: {'input': 'What is black body radiation?'}

尝试输入不同的内容。

In [ ]:
chain.run("what is 2 + 2")

LLM输出：math: {'input': 'what is 2 + 2'}
> Finished chain.
The answer to 2 + 2 is 4.

尝试输入不同的内容。如果不是四个学科的时候，会发生什么？以生物问题为例。

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

如果运行一些LLM判断不了的对话，那又会发生什么？

In [ ]:
chain.run("你是谁?")

LLM输出：None: {'input': '你是谁'}

我是一个人工智能助手，可以回答你的问题和提供帮助。有什么可以帮到你的吗？